In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import ssl
import urllib
import urllib3
import re
from time import sleep
import ssl
import os
from tqdm import tqdm
import random

ssl._create_default_https_context = ssl._create_unverified_context
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# class_names_eng = ['blouse', 'cardigan', 'coat', 'hoodie', 'jacket', 'jumper','jumpsuit',
#                    'knit', 'longpants', 'onepiece', 'shirt', 'shortpants', 'skirt', 'tshirt']
# class_names_kor = ['블라우스', '가디건', '코트', '후드티', '자켓', '점퍼', '점프수트',
#                    '니트', '긴바지', '원피스', '셔츠', '반바지', '치마', '티셔츠']
# color_list = ["블랙", "화이트", "레드", "블루", "옐로우", "그린", "퍼플", "브라운", "네이비", "오렌지", "그레이", "핑크", "베이지"]

# 14개
dict_class_name = {
    '블라우스' : 'blouse',
    '가디건' : 'cardigan',
    '코트' : 'coat',
    '후드티' : 'hoodie',
    '자켓' : 'jacket',
    '점퍼' : 'jumper',
    '점프수트' : 'jumpsuit',
    '니트' : 'knit',
    '긴바지' : 'longpants',
    '원피스' : 'onepiece',
    '셔츠' : 'shirt',
    '반바지' : 'shortpants',
    '치마' : 'skirt',
    '티셔츠' : 'tshirt'    
}

# 13개
dict_color_name = {
    "블랙" : "black",
    "화이트" : "white",
    "레드" : "red",
    "블루" : "blue",
    "옐로우" : "yellow",
    "그린" : "green",
    "퍼플" : "purple",
    "브라운" : "brown",
    "네이비" : "navy",
    "오렌지" : "orange",
    "그레이" : "gray",
    "핑크" : "pink",
    "베이지" : "beige"
}

dict_filter_color = {
    "블랙" : "1%23attr_10496%244278%40DEFAULT",
    "화이트" : "1%23attr_10496%244289%40DEFAULT",
    "레드" : "1%23attr_10496%244282%40DEFAULT",
    "블루" : "1%23attr_10496%244286%40DEFAULT",
    "옐로우" : "1%23attr_10496%244284%40DEFAULT",
    "그린" : "1%23attr_10496%244285%40DEFAULT",
    "퍼플" : "1%23attr_10496%244287%40DEFAULT",
    "브라운" : "1%23attr_10496%244290%40DEFAULT",
    "네이비" : "1%23attr_10496%244279%40DEFAULT",
    "오렌지" : "1%23attr_10496%244283%40DEFAULT",
    "그레이" : "1%23attr_10496%244280%40DEFAULT",
    "핑크" : "1%23attr_10496%244288%40DEFAULT",
    "베이지" : "1%23attr_10496%244292%40DEFAULT"
}

In [3]:
filter_page_num = 1
filter_list_size = 72
download_folder = 'img_data'
new_img_size = 550

In [4]:
def create_directory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [5]:
# 다운로드 폴더 생성

create_directory(download_folder)

for class_name_kor, class_name_eng in dict_class_name.items():
    create_directory(download_folder +'/'+ class_name_eng)
    
    for color_name_kor, color_name_eng in dict_color_name.items():
        create_directory(download_folder +'/'+ class_name_eng +'/'+ color_name_eng)

In [ ]:
ua = UserAgent(verify_ssl=False)
user_agent = ua.random
    
for class_name_kor, class_name_eng in dict_class_name.items():
    for color_name_kor, color_name_eng in dict_color_name.items():

        url = f'https://www.coupang.com/np/search?component=' \
            + f'&q={class_name_kor}' \
            + f'&page={filter_page_num}' \
            + f'&listSize={filter_list_size}' \
            + f'&filter={dict_filter_color[color_name_kor]}'
        print(class_name_kor, color_name_kor)
        print(url)
        

        headers = {"User-Agent": user_agent, 
                  "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}

        res = requests.get(url, headers=headers, verify=False)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml") 

        titles = soup.select('#productList > li')
        print(f'검색결과: {len(titles)}')
        
        # ---------------------------------------------------------------------
        
        number = 0
        
        download_path = download_folder +'/'+ class_name_eng +'/'+ color_name_eng
#         print(download_path)
        num_exist_files = len(os.listdir(download_path))
        
        if num_exist_files == filter_list_size:
            print(f"!!MSG : {num_exist_files} files exist. PASS. ({class_name_kor}, {color_name_kor})")
        else:
            
            for title in tqdm(titles):
                number = number + 1

                # 상품명
                title_str = title.select_one('a > dl > dd > div > div.name').text
#                 print(title_str)

                # 이미지 링크
                product_title = title.select_one('a > dl > dt > img')['src']
                if product_title == "//img1a.coupangcdn.com/image/coupang/search/blank1x1.gif":
                    product_title = title.select_one('a > dl > dt > img')['data-img-src']
                product_title = re.sub('\n','', product_title)
                product_title = product_title.lstrip()
                img_url = 'https:'+product_title
#                 print('[' + str(number) + ']', img_url)
                
                split_url = img_url.split('230x230ex')
                new_img_url = split_url[0] + f'{new_img_size}x{new_img_size}ex' + split_url[1]

                try:
                    urllib.request.urlretrieve(new_img_url, f"{download_path}/{class_name_eng}_{color_name_eng}_{number}.jpg")
                except:
                    print(f"!!ERR : {class_name_kor},{color_name_kor}")
                sleep(random.uniform(0.1, 0.5))

        sleep(1)
#         break
#     break

블라우스 블랙
https://www.coupang.com/np/search?component=&q=블라우스&page=1&listSize=72&filter=1%23attr_10496%244278%40DEFAULT
검색결과: 72


100%|███████████████████████████████████████████████████████████████| 72/72 [00:05<00:00, 12.06it/s]


블라우스 화이트
https://www.coupang.com/np/search?component=&q=블라우스&page=1&listSize=72&filter=1%23attr_10496%244289%40DEFAULT
검색결과: 72


100%|███████████████████████████████████████████████████████████████| 72/72 [00:06<00:00, 11.90it/s]


블라우스 레드
https://www.coupang.com/np/search?component=&q=블라우스&page=1&listSize=72&filter=1%23attr_10496%244282%40DEFAULT
검색결과: 72


...

100%|███████████████████████████████████████████████████████████████| 71/71 [00:17<00:00,  4.04it/s]


긴바지 블루
https://www.coupang.com/np/search?component=&q=긴바지&page=1&listSize=72&filter=1%23attr_10496%244286%40DEFAULT
검색결과: 72


 60%|█████████████████████████████████████▋                         | 43/72 [00:08<00:05,  5.54it/s]

!!ERR : 긴바지,블루


 72%|█████████████████████████████████████████████▌                 | 52/72 [00:11<00:04,  4.48it/s]

In [ ]:

product_img_url = "http:"+ soup.select_one('#repImageContainer > img')['src']
urllib.request.urlretrieve(product_img_url, "이미지.jpg")

In [10]:
len(os.listdir('img_data/blouse/black'))


72